# 0.04 Verif Anomalies

---

Generate anomalies for the verification data.

In [1]:
%load_ext lab_black

import numpy as np
import xarray as xr
import glob
from dask.distributed import Client

In [2]:
client = Client("tcp://10.12.205.11:42722")

In [3]:
filepath = "/glade/work/jaye/climpred/data/"
filelist = np.sort(glob.glob(filepath + "T2AVG_*.nc"))

In [4]:
ds = xr.open_mfdataset(filelist, combine="nested", parallel=True, concat_dim="time")

I'm just going to remove a daily climatology. Ideally you would follow similar steps to the S2S anomaly generation and do the bandpass smoothing on the climatology.

In [5]:
clim = ds.groupby("time.dayofyear").mean("time")

In [6]:
anom = ds.groupby("time.dayofyear") - clim

In [7]:
anom = anom.chunk({"lat": -1, "lon": -1, "time": "auto"}).rename({"T2AVG": "TAS"})

In [8]:
%time anom.to_zarr('/glade/scratch/rbrady/S2S/ERA5.tas.zarr', consolidated=True)

CPU times: user 3.88 s, sys: 202 ms, total: 4.08 s
Wall time: 14.4 s
